<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv


In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [ ]:
# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

# 계절 분류 함수 정의
def classify_season(month):
    if  month == 1 or month == 2 or month == 12:
        return '겨울'
    elif 3 <= month <= 5:
        return '봄'
    elif 6 <= month <= 8:
        return '여름'
    else:
        return '가을'

# '일시' 컬럼에서 월 정보를 추출하여 '계절' 열 추가
data['월'] = data['일시'].dt.month
data['계절'] = data['월'].apply(classify_season)

# 계절별 중앙값 계산
seasonal_medians = data.groupby('계절').transform('median')

# 결측값 대체
data.fillna(seasonal_medians, inplace=True)

# print(seasonal_medians)

<ipython-input-4-cf72f31a3d80>:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seasonal_medians = data.groupby('계절').transform('median')


In [ ]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]
# 일시를 제외해보기로 함
target = data['평균기온']

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
poly.fit(train)
train_poly = poly.transform(train)

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_poly)
train_scaled = ss.transform(train_poly)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly, target)

# print(lr.score(train_poly, target))

from sklearn.metrics import mean_absolute_error
lr_pred = lr.predict(train_poly)

lr_mae = mean_absolute_error(target, lr_pred)

print("훈련 세트 MAE:", lr_mae)

submission['평균기온'] = lr_pred[:358]
submission.to_csv("DEAGYUL_9_submission.csv", index=False)

print(submission.head(100))

훈련 세트 MAE: 0.36509673215087207
            일시       평균기온
0   2023-01-01  -1.889259
1   2023-01-02  -2.393677
2   2023-01-03   3.412295
3   2023-01-04   5.895525
4   2023-01-05  -3.796310
..         ...        ...
95  2023-04-06   5.227001
96  2023-04-07   9.074437
97  2023-04-08   5.015264
98  2023-04-09   7.723261
99  2023-04-10  11.841754

[100 rows x 2 columns]
